# PROGETTO CUSTOMER

## Customer Analytics – Data Preparation & Customer 360

### Dataset
- **Online Retail Dataset** (transaction-level)
- Una riga rappresenta una singola transazione di prodotto
- Il `CustomerID` non è univoco a livello di riga

---

### Obiettivo di questo step
Costruire una **vista cliente (Customer 360)**:
- una riga per cliente
- `CustomerID` univoco
- feature comportamentali sintetiche
- base per analisi di churn, segmentazione o propensity

---

### Passaggi effettuati

1. **Gestione delle date**
   - Conversione esplicita di `InvoiceDate` in formato `datetime64[ns]`
   - Verifica del tipo per garantire operazioni temporali corrette
     

2. **Pulizia dei dati**
   - Rimozione delle righe senza `CustomerID`
   - Conversione del `CustomerID` in formato intero
   - Creazione della variabile `revenue = Quantity × UnitPrice`
   - Mantenimento dei resi come segnale comportamentale


3. **Definizione della snapshot date**
   - Scelta di una data di riferimento per separare:
     - **storico** (feature)
     - **futuro** (target)
   - Prevenzione del data leakage
     

4. **Filtraggio dello storico**
   - Utilizzo esclusivo delle transazioni precedenti alla snapshot date
   - Dataset pronto per il feature engineering
     

5. **Aggregazione a livello cliente**
   - Group by `CustomerID`
   - Costruzione di feature come:
     - data primo e ultimo acquisto
     - numero di fatture
     - fatturato totale
     - numero di prodotti distinti


6. **Feature temporali derivate**
   - **Recency**: giorni dall’ultimo acquisto
   - **Tenure**: giorni dalla prima interazione


7. **Controlli finali**
   - Verifica che `CustomerID` sia univoco
   - Assenza di valori mancanti
   - Dataset pronto per modeling

---

### Output
- Tabella **customer-level (Customer 360)**
- Una riga per cliente
- Feature interpretabili e riutilizzabili
- Pronta per churn prediction, segmentazione o analisi avanzate


## 1) Importazione dataset 

In [6]:
import pandas as pd

url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx"

df = pd.read_excel(url)

In [14]:
df[df["CustomerID"] == df["CustomerID"].iloc[0]].head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


## 2) Trasformazione del dataset in una vista customer-level aggregata

### STEP 1 — Conversione esplicita delle date

In [16]:
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

In [17]:
df["InvoiceDate"].dtype

dtype('<M8[ns]')

### STEP 2 — Gestione dei clienti mancanti

In [22]:
df = df.dropna(subset=["CustomerID"])

In [23]:
df["CustomerID"] = df["CustomerID"].astype(int)

In [25]:
df["CustomerID"].isna().sum()

np.int64(0)

### STEP 3 — Gestione resi e creazione revenue
Nel fashion:
__Quantity < 0 = reso__

Creiamo la colonna __revenue__

In [26]:
df["revenue"] = df["Quantity"] * df["UnitPrice"]

### STEP 4 — Definizione della snapshot date 

La snapshot date è una data fissa che separa passato (feature) e futuro (target)


In [27]:
snapshot_date = pd.Timestamp("2011-09-01")

In [28]:
df["InvoiceDate"].min(), df["InvoiceDate"].max()

(Timestamp('2010-12-01 08:26:00'), Timestamp('2011-12-09 12:50:00'))

### STEP 5 — Teniamo solo lo storico (prima della snapshot)

In [29]:
df_hist = df[df["InvoiceDate"] < snapshot_date].copy()

### STEP 6 — Aggregazione per CustomerID

Ora trasformiamo
molte righe per cliente
in
una riga per cliente

In [31]:
customer_360 = (
    df_hist
    .groupby("CustomerID")
    .agg(
        first_purchase_date=("InvoiceDate", "min"),
        last_purchase_date=("InvoiceDate", "max"),
        n_invoices=("InvoiceNo", "nunique"),
        total_revenue=("revenue", "sum"),
        avg_revenue_per_line=("revenue", "mean"),
        n_products=("StockCode", "nunique")
    )
    .reset_index()
)

### STEP 7 — Feature temporali derivate 

__Recency__ (da quanto tempo non compra)

__Tenure__ (da quanto tempo è cliente)


In [33]:
customer_360["recency_days"] = (
    snapshot_date - customer_360["last_purchase_date"]
).dt.days


In [34]:
customer_360["tenure_days"] = (
    snapshot_date - customer_360["first_purchase_date"]
).dt.days


### STEP 8 —  Return rate

In [36]:
returns = (
    df_hist[df_hist["Quantity"] < 0]
    .groupby("CustomerID")["Quantity"]
    .count()
)

purchases = (
    df_hist[df_hist["Quantity"] > 0]
    .groupby("CustomerID")["Quantity"]
    .count()
)

customer_360["return_rate"] = (
    returns / purchases
).fillna(0)

### STEP 9 —  Controlli finali

In [38]:
customer_360["CustomerID"].is_unique

True

In [39]:
customer_360.shape

(3360, 10)

In [40]:
customer_360.head()

,CustomerID,first_purchase_date,last_purchase_date,n_invoices,total_revenue,avg_revenue_per_line,n_products,recency_days,tenure_days,return_rate
0,12346,2011-01-18 10:01:00,2011-01-18 10:17:00,2,0.00,0.000000,1,225,225,NaN
1,12347,2010-12-07 14:57:00,2011-08-02 08:48:00,5,2790.86,22.506935,82,29,267,NaN
2,12348,2010-12-16 19:09:00,2011-04-05 10:47:00,3,1487.24,53.115714,22,148,258,NaN
3,12350,2011-02-02 16:01:00,2011-02-02 16:01:00,1,334.40,19.670588,17,210,210,NaN
4,12352,2011-02-16 12:33:00,2011-03-22 16:08:00,8,601.18,12.524583,26,162,196,NaN


### STEP 10 - Salvataggio dataset customer aggregato 

In [44]:
customer_360.to_excel(
    "customer_360_snapshot_2011-09-01.xlsx",
    index=False
)

## 3) Costruzione modello del churn Regressione Logistica

### Step 0 — Import e controlli minimi
 assicuriamoci che le colonne chiave esistano e che i tipi siano corretti.

In [48]:
import numpy as np

In [49]:
required_cols = {"CustomerID", "InvoiceDate", "InvoiceNo", "Quantity", "UnitPrice"}
missing = required_cols - set(df.columns)
assert not missing, f"Mancano colonne nel dataframe df: {missing}"

# Assicura che InvoiceDate sia datetime
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"], errors="coerce")
assert df["InvoiceDate"].isna().sum() == 0, "InvoiceDate contiene valori non convertibili a datetime"

# Assicura che CustomerID non abbia NA
assert df["CustomerID"].isna().sum() == 0, "CustomerID contiene ancora valori mancanti"

### Step 1 — Definisci il churn e crea le label (target)

Scegli una finestra futura (es. 90 giorni).

Se un cliente fa almeno un acquisto in quella finestra → non churn (churn = 0)

Se non compra → churn (churn = 1)


In [51]:
future_window_days = 90
snapshot_date = pd.Timestamp("2011-09-01") 

# Considero acquisto solo quantity > 0 (evita che un "reso" conti come attività)
df_purchases = df[df["Quantity"] > 0].copy()

df_future = df_purchases[
    (df_purchases["InvoiceDate"] >= snapshot_date) &
    (df_purchases["InvoiceDate"] < snapshot_date + pd.Timedelta(days=future_window_days))
].copy()

future_buyers = df_future["CustomerID"].unique()

# Creo churn: 1 se NON compra nel futuro, 0 se compra
customer_360 = customer_360.copy()
customer_360["churn"] = (~customer_360["CustomerID"].isin(future_buyers)).astype(int)

# Check distribuzione (sbilanciamento realistico)
customer_360["churn"].value_counts(dropna=False), customer_360["churn"].mean()

(churn
 0    1876
 1    1484
 Name: count, dtype: int64,
 np.float64(0.44166666666666665))

### Step 2 — Seleziona feature e prepara X, y

Usiamo solo feature disponibili alla snapshot (quindi calcolate sullo storico).

In [55]:
target_col = "churn"

feature_cols = [
    "recency_days",
    "tenure_days",
    "n_invoices",
    "total_revenue",
    "avg_revenue_per_line",
    "n_products",
    "return_rate",
]

missing_feat = set(feature_cols) - set(customer_360.columns)
assert not missing_feat, f"Mancano feature in customer_360: {missing_feat}"

X = customer_360[feature_cols].copy()
y = customer_360[target_col].copy()

X = X.fillna(0)

X.shape, y.value_counts()

((3360, 7),
 churn
 0    1876
 1    1484
 Name: count, dtype: int64)

### Step 3 — Train/test split

Con una sola snapshot, usiamo uno split stratificato per mantenere la stessa proporzione churn/non churn in train e test.

In [56]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42,
    stratify=y
)

y_train.mean(), y_test.mean()

(np.float64(0.44166666666666665), np.float64(0.44166666666666665))

### Step 4 — Modello baseline interpretabile (Logistic Regression)

La regressione logistica è:

interpretabile

veloce

ottima come baseline

In [57]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

clf = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("model", LogisticRegression(max_iter=2000, class_weight="balanced", random_state=42))
])

clf.fit(X_train, y_train)

,steps,"[('scaler', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,copy,True
,with_mean,True
,with_std,True
,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0


### Step 5 — Predizioni e metriche


__5.1 Probabilità di churn__

In [58]:
y_prob = clf.predict_proba(X_test)[:, 1]

__5.2 AUC__

In [59]:
from sklearn.metrics import roc_auc_score

auc = roc_auc_score(y_test, y_prob)
auc

0.7046534750199713

__5.3 Precision/Recall con soglia standard 0.5__

In [61]:
from sklearn.metrics import classification_report

y_pred_05 = (y_prob >= 0.5).astype(int)
print(classification_report(y_test, y_pred_05, digits=3))

              precision    recall  f1-score   support

           0      0.737     0.580     0.649       469
           1      0.582     0.739     0.651       371

    accuracy                          0.650       840
   macro avg      0.659     0.659     0.650       840
weighted avg      0.668     0.650     0.650       840



### Step 6 — Metrica “business”: Recall@k

In [62]:
def recall_at_k(y_true: pd.Series, y_score: np.ndarray, k: float = 0.10) -> float:
    """
    y_true: serie 0/1 con indice allineato al test set
    y_score: probabilità stimate dal modello
    k: quota di clienti da contattare (0.10 = 10%)
    """
    y_true = pd.Series(y_true).reset_index(drop=True)
    y_score = pd.Series(y_score).reset_index(drop=True)

    cutoff = max(1, int(len(y_score) * k))
    top_idx = np.argsort(y_score)[-cutoff:]
    return y_true.iloc[top_idx].sum() / max(1, y_true.sum())

for k in [0.05, 0.10, 0.20]:
    print(k, recall_at_k(y_test, y_prob, k=k))

0.05 0.07008086253369272
0.1 0.1347708894878706
0.2 0.2749326145552561


### Step 7 — Interpretazione: quali feature spingono il churn?


In [63]:
model = clf.named_steps["model"]
coefs = pd.Series(model.coef_[0], index=feature_cols).sort_values()

coefs

n_invoices             -0.973734
n_products             -0.513550
tenure_days            -0.160027
total_revenue          -0.142404
avg_revenue_per_line   -0.007045
return_rate             0.000000
recency_days            0.498526
dtype: float64

### Step 8 — Scoring su tutti i clienti e lista operativa

In [68]:
customer_360["churn_probability"] = clf.predict_proba(X)[:, 1]
ranking = customer_360.sort_values("churn_probability", ascending=False)

ranking[["CustomerID", "churn_probability", "churn"]].head(10)

,CustomerID,churn_probability,churn
1074,14213,0.832481,1
2639,16995,0.830950,1
811,13747,0.830634,1
1294,14576,0.830528,1
1351,14679,0.830350,1
2932,17547,0.829897,1
1567,15070,0.829710,1
2743,17181,0.829199,0
276,12791,0.828906,1
148,12586,0.828661,0


__top 10%__

In [66]:
top_k = int(len(ranking) * 0.10)
to_contact = ranking.head(top_k).copy()

to_contact[["CustomerID", "churn_probability"]].head()

,CustomerID,churn_probability
1074,14213,0.832481
2639,16995,0.830950
811,13747,0.830634
1294,14576,0.830528
1351,14679,0.830350


### STEP 9 - Creazione report con tutte le metriche del modello train e test

In [73]:
y_prob_train = clf.predict_proba(X_train)[:, 1]
y_prob_test  = clf.predict_proba(X_test)[:, 1]

y_pred_train = (y_prob_train >= 0.5).astype(int)
y_pred_test  = (y_prob_test  >= 0.5).astype(int)

In [74]:
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score

def compute_metrics(y_true, y_prob, threshold=0.5):
    y_pred = (y_prob >= threshold).astype(int)
    return {
        "roc_auc": roc_auc_score(y_true, y_prob),
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
    }

In [75]:
metrics_train = compute_metrics(y_train, y_prob_train)
metrics_test  = compute_metrics(y_test,  y_prob_test)

comparison_df = pd.DataFrame([metrics_train, metrics_test],
                             index=["train", "test"])

comparison_df

,roc_auc,precision,recall,f1
train,0.759061,0.619220,0.769991,0.686424
test,0.704653,0.581741,0.738544,0.650831


In [76]:
def recall_at_k(y_true, y_score, k=0.10):
    y_true = pd.Series(y_true).reset_index(drop=True)
    y_score = pd.Series(y_score).reset_index(drop=True)
    cutoff = max(1, int(len(y_score) * k))
    top_idx = np.argsort(y_score)[-cutoff:]
    return y_true.iloc[top_idx].sum() / max(1, y_true.sum())

recall_k_df = pd.DataFrame({
    "train": [recall_at_k(y_train, y_prob_train, k=0.10)],
    "test":  [recall_at_k(y_test,  y_prob_test,  k=0.10)]
}, index=["recall_at_10pct"])

recall_k_df

,train,test
recall_at_10pct,0.167116,0.134771


In [77]:
from sklearn.inspection import permutation_importance

perm = permutation_importance(
    clf,
    X_test,
    y_test,
    scoring="roc_auc",
    n_repeats=10,
    random_state=42
)

perm_df = (
    pd.DataFrame({
        "feature": X_test.columns,
        "importance_drop_in_auc": perm.importances_mean
    })
    .sort_values("importance_drop_in_auc", ascending=False)
)

perm_df.head(10)

,feature,importance_drop_in_auc
2,n_invoices,0.040180
0,recency_days,0.030004
5,n_products,0.015805
1,tenure_days,0.005757
3,total_revenue,0.000379
6,return_rate,0.000000
4,avg_revenue_per_line,-0.000053


In [78]:
with pd.ExcelWriter("train_test_analysis.xlsx") as writer:
    comparison_df.to_excel(writer, sheet_name="metrics_train_vs_test")
    recall_k_df.to_excel(writer, sheet_name="recall_at_k")
    perm_df.to_excel(writer, sheet_name="perm_importance_test")

## 4) Modello xgBoost con validation set

In [79]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 7.5 MB/s eta 0:00:00a 0:00:01


In [80]:
from sklearn.model_selection import train_test_split

# 60% train, 20% validation, 20% test
X_train, X_tmp, y_train, y_tmp = train_test_split(
    X, y,
    test_size=0.40,
    stratify=y,
    random_state=42
)

X_val, X_test, y_val, y_test = train_test_split(
    X_tmp, y_tmp,
    test_size=0.50,
    stratify=y_tmp,
    random_state=42
)

In [81]:
y_train.mean(), y_val.mean(), y_test.mean()

(np.float64(0.44146825396825395),
 np.float64(0.4419642857142857),
 np.float64(0.4419642857142857))

In [82]:
from xgboost import XGBClassifier

xgb = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=4,
    subsample=0.9,
    colsample_bytree=0.9,
    scale_pos_weight=(len(y_train) - y_train.sum()) / y_train.sum(),
    random_state=42,
    n_jobs=-1,
    eval_metric="auc"
)

xgb.fit(X_train, y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.9
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'auc'


### Validazione su valuation set

In [84]:
y_prob_train = xgb.predict_proba(X_train)[:, 1]
y_prob_val   = xgb.predict_proba(X_val)[:, 1]

y_pred_train = (y_prob_train >= 0.5).astype(int)
y_pred_val   = (y_prob_val   >= 0.5).astype(int)

In [85]:
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    precision_score,
    recall_score,
    f1_score
)

def compute_metrics(y_true, y_prob, threshold=0.5):
    y_pred = (y_prob >= threshold).astype(int)
    return {
        "roc_auc": roc_auc_score(y_true, y_prob),
        "pr_auc": average_precision_score(y_true, y_prob),
        "precision@0.5": precision_score(y_true, y_pred, zero_division=0),
        "recall@0.5": recall_score(y_true, y_pred, zero_division=0),
        "f1@0.5": f1_score(y_true, y_pred, zero_division=0),
    }

In [86]:
metrics_train = compute_metrics(y_train, y_prob_train)
metrics_val   = compute_metrics(y_val,   y_prob_val)

metrics_df = pd.DataFrame(
    [metrics_train, metrics_val],
    index=["train", "validation"]
)

metrics_df

,roc_auc,pr_auc,precision@0.5,recall@0.5,f1@0.5
train,0.963791,0.953090,0.838384,0.932584,0.882979
validation,0.723385,0.614261,0.617834,0.653199,0.635025


In [87]:
def recall_at_k(y_true, y_score, k=0.10):
    y_true = pd.Series(y_true).reset_index(drop=True)
    y_score = pd.Series(y_score).reset_index(drop=True)
    cutoff = max(1, int(len(y_score) * k))
    top_idx = np.argsort(y_score)[-cutoff:]
    return float(y_true.iloc[top_idx].sum() / max(1, y_true.sum()))

recall_k_df = pd.DataFrame({
    "recall_at_10pct": [
        recall_at_k(y_train, y_prob_train, k=0.10),
        recall_at_k(y_val,   y_prob_val,   k=0.10)
    ]
}, index=["train", "validation"])

recall_k_df

,recall_at_10pct
train,0.224719
validation,0.151515


In [93]:
from xgboost import XGBClassifier
from xgboost.callback import EarlyStopping

In [95]:
xgb = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=2,
    min_child_weight=5,
    subsample=0.7,
    colsample_bytree=0.7,
    reg_lambda=5.0,
    reg_alpha=1.0,
    random_state=42,
    n_jobs=-1,
    eval_metric="auc"¢¢
)

In [98]:
xgb.fit(X_train, y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.7
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'auc'


In [99]:
y_prob_train = xgb.predict_proba(X_train)[:, 1]
y_prob_val   = xgb.predict_proba(X_val)[:, 1]

In [102]:
def compute_metrics(y_true, y_prob, threshold=0.5):
    y_pred = (y_prob >= threshold).astype(int)
    return {
        "roc_auc": roc_auc_score(y_true, y_prob),
        "pr_auc": average_precision_score(y_true, y_prob),
        "precision@0.5": precision_score(y_true, y_pred, zero_division=0),
        "recall@0.5": recall_score(y_true, y_pred, zero_division=0),
        "f1@0.5": f1_score(y_true, y_pred, zero_division=0),
    }

metrics_train = compute_metrics(y_train, y_prob_train)
metrics_val   = compute_metrics(y_val,   y_prob_val)

comparison_df = pd.DataFrame(
    [metrics_train, metrics_val],
    index=["train", "validation"]
)

comparison_df

,roc_auc,pr_auc,precision@0.5,recall@0.5,f1@0.5
train,0.822764,0.772286,0.698997,0.704494,0.701735
validation,0.742932,0.627239,0.643357,0.619529,0.631218


In [103]:
y_prob_test = xgb.predict_proba(X_test)[:, 1]
y_pred_test = (y_prob_test >= 0.5).astype(int)

In [104]:
from sklearn.metrics import (
    roc_auc_score,
    average_precision_score,
    precision_score,
    recall_score,
    f1_score
)

test_metrics = {
    "roc_auc": roc_auc_score(y_test, y_prob_test),
    "pr_auc": average_precision_score(y_test, y_prob_test),
    "precision@0.5": precision_score(y_test, y_pred_test, zero_division=0),
    "recall@0.5": recall_score(y_test, y_pred_test, zero_division=0),
    "f1@0.5": f1_score(y_test, y_pred_test, zero_division=0),
}

test_metrics

{'roc_auc': 0.6880359147025814,
 'pr_auc': 0.5751277198647353,
 'precision@0.5': 0.578397212543554,
 'recall@0.5': 0.5589225589225589,
 'f1@0.5': 0.5684931506849316}

In [105]:
def recall_at_k(y_true, y_score, k=0.10):
    y_true = pd.Series(y_true).reset_index(drop=True)
    y_score = pd.Series(y_score).reset_index(drop=True)

    cutoff = max(1, int(len(y_score) * k))
    top_idx = np.argsort(y_score)[-cutoff:]
    return float(y_true.iloc[top_idx].sum() / max(1, y_true.sum()))

recall_test_10pct = recall_at_k(y_test, y_prob_test, k=0.10)
recall_test_10pct

0.13131313131313133

In [106]:
final_comparison = pd.DataFrame(
    [
        metrics_train,
        metrics_val,
        test_metrics
    ],
    index=["train", "validation", "test"]
)

final_comparison

,roc_auc,pr_auc,precision@0.5,recall@0.5,f1@0.5
train,0.822764,0.772286,0.698997,0.704494,0.701735
validation,0.742932,0.627239,0.643357,0.619529,0.631218
test,0.688036,0.575128,0.578397,0.558923,0.568493


### Creazione report

In [108]:
# =========================
# 0) Utility: metriche
# =========================
def compute_metrics(y_true, y_prob, threshold=0.5):
    y_pred = (np.asarray(y_prob) >= threshold).astype(int)
    return {
        "roc_auc": roc_auc_score(y_true, y_prob),
        "pr_auc": average_precision_score(y_true, y_prob),
        "precision@0.5": precision_score(y_true, y_pred, zero_division=0),
        "recall@0.5": recall_score(y_true, y_pred, zero_division=0),
        "f1@0.5": f1_score(y_true, y_pred, zero_division=0),
    }

def recall_at_k(y_true, y_score, k=0.10):
    y_true = pd.Series(y_true).reset_index(drop=True)
    y_score = pd.Series(y_score).reset_index(drop=True)
    cutoff = max(1, int(len(y_score) * k))
    top_idx = np.argsort(y_score)[-cutoff:]
    return float(y_true.iloc[top_idx].sum() / max(1, y_true.sum()))

# =========================
# 1) Scoring su customer_360
# =========================
# Usiamo le stesse colonne viste dal modello (X_train.columns)
feature_cols = list(X_train.columns)

X_all = customer_360[feature_cols].copy().fillna(0)

ranking = customer_360.copy()
ranking["churn_probability"] = xgb.predict_proba(X_all)[:, 1]

# Ordina per rischio decrescente
ranking = ranking.sort_values("churn_probability", ascending=False).reset_index(drop=True)

# =========================
# 2) Selezione "high risk"
#    Scegli UNA modalità:
#    A) top_pct (consigliata)
#    B) soglia assoluta
# =========================

# --- Modalità A: Top % clienti (consigliata per scenario operativo)
top_pct = 0.10  # 10% più a rischio
top_k = max(1, int(len(ranking) * top_pct))
churn_high_risk = ranking.head(top_k).copy()

# --- Modalità B: soglia assoluta (se preferisci)
# threshold = 0.70
# churn_high_risk = ranking[ranking["churn_probability"] >= threshold].copy()

# (Consiglio) tieni solo colonne utili nel foglio operativo
cols_operativi = ["CustomerID", "churn_probability"]
# aggiungi colonne se esistono
for c in ["recency_days", "tenure_days", "n_invoices", "total_revenue", "n_products", "return_rate"]:
    if c in churn_high_risk.columns:
        cols_operativi.append(c)

churn_high_risk = churn_high_risk[cols_operativi].copy()

# =========================
# 3) Metriche train / val / test
# =========================
rows = []

# Train
y_prob_train = xgb.predict_proba(X_train)[:, 1]
m_train = compute_metrics(y_train, y_prob_train, threshold=0.5)
m_train["recall_at_10pct"] = recall_at_k(y_train, y_prob_train, k=0.10)
m_train["dataset"] = "train"
rows.append(m_train)

# Validation (solo se esiste in memoria)
if "X_val" in globals() and "y_val" in globals() and X_val is not None and y_val is not None:
    y_prob_val = xgb.predict_proba(X_val)[:, 1]
    m_val = compute_metrics(y_val, y_prob_val, threshold=0.5)
    m_val["recall_at_10pct"] = recall_at_k(y_val, y_prob_val, k=0.10)
    m_val["dataset"] = "validation"
    rows.append(m_val)

# Test
y_prob_test = xgb.predict_proba(X_test)[:, 1]
m_test = compute_metrics(y_test, y_prob_test, threshold=0.5)
m_test["recall_at_10pct"] = recall_at_k(y_test, y_prob_test, k=0.10)
m_test["dataset"] = "test"
rows.append(m_test)

metrics_df = pd.DataFrame(rows)[
    ["dataset", "roc_auc", "pr_auc", "precision@0.5", "recall@0.5", "f1@0.5", "recall_at_10pct"]
]

# =========================
# 4) Permutation importance (sul TEST)
# =========================
perm = permutation_importance(
    xgb,
    X_test,
    y_test,
    scoring="roc_auc",
    n_repeats=10,
    random_state=42
)

perm_df = (
    pd.DataFrame({
        "feature": X_test.columns,
        "importance_drop_in_auc": perm.importances_mean
    })
    .sort_values("importance_drop_in_auc", ascending=False)
    .reset_index(drop=True)
)

# =========================
# 5) Scrittura Excel multi-sheet
# =========================
output_file = "churn_analysis_output.xlsx"

with pd.ExcelWriter(output_file) as writer:
    churn_high_risk.to_excel(writer, sheet_name="churn_high_risk", index=False)
    metrics_df.to_excel(writer, sheet_name="model_metrics", index=False)
    perm_df.to_excel(writer, sheet_name="feature_importance", index=False)

print(f"Creato file: {output_file}")
print("Sheet inclusi: churn_high_risk, model_metrics, feature_importance")


Creato file: churn_analysis_output.xlsx
Sheet inclusi: churn_high_risk, model_metrics, feature_importance


## 5) Customer Lifetime Value (CLV)

CLV ≈ spesa media per periodo × frequenza × durata attesa

In [109]:
import numpy as np
import pandas as pd

# --- Parametri CLV
clv_horizon_months = 12  # orizzonte del CLV: 12 mesi

# --- Copia per sicurezza
clv_df = customer_360.copy()

# 1) Tenure in mesi (evita divisione per zero)
# 30.44 ≈ giorni medi per mese
clv_df["tenure_months"] = (clv_df["tenure_days"] / 30.44).clip(lower=1)

# 2) Spesa media mensile storica (proxy semplice)
clv_df["avg_monthly_revenue"] = (clv_df["total_revenue"] / clv_df["tenure_months"]).clip(lower=0)

# 3) CLV proxy a 12 mesi
clv_df["clv_12m"] = clv_df["avg_monthly_revenue"] * clv_horizon_months

# 4) (Opzionale, consigliato) CLV “risk-adjusted” usando la probabilità di churn
# Se hai una colonna churn_probability (da qualsiasi modello), la usiamo.
if "churn_probability" in clv_df.columns:
    clv_df["clv_12m_adj"] = clv_df["clv_12m"] * (1 - clv_df["churn_probability"])
else:
    clv_df["clv_12m_adj"] = clv_df["clv_12m"]

# 5) Gestione outlier (molto comune): clip al 99° percentile
p99 = clv_df["clv_12m_adj"].quantile(0.99)
clv_df["clv_12m_adj_clipped"] = clv_df["clv_12m_adj"].clip(upper=p99)

# 6) Bucket di valore (basso/medio/alto) con quantili
clv_df["clv_bucket"] = pd.qcut(clv_df["clv_12m_adj_clipped"], q=3, labels=["low", "mid", "high"])

# 7) Tabella finale da esportare
cols = ["CustomerID", "clv_12m", "clv_12m_adj", "clv_12m_adj_clipped", "clv_bucket"]
if "churn_probability" in clv_df.columns:
    cols.insert(1, "churn_probability")

customer_clv = clv_df[cols].sort_values("clv_12m_adj_clipped", ascending=False).reset_index(drop=True)

customer_clv.head(10)

,CustomerID,churn_probability,clv_12m,clv_12m_adj,clv_12m_adj_clipped,clv_bucket
0,13408,6.104228e-05,22872.348396,22870.952215,22355.124126,high
1,12931,4.764769e-02,35515.662724,33823.423499,22355.124126,high
2,13798,2.559013e-04,36761.550900,36752.143571,22355.124126,high
3,14646,1.779092e-07,252588.157493,252588.112555,22355.124126,high
4,16333,6.188056e-02,25113.786488,23559.731238,22355.124126,high
5,17404,5.135170e-02,27587.491015,26170.826370,22355.124126,high
6,14298,1.029854e-05,50003.797366,50003.282400,22355.124126,high
7,17450,7.725146e-04,114782.550040,114693.878849,22355.124126,high
8,16029,9.175152e-05,48762.993389,48758.519310,22355.124126,high
9,15311,3.631085e-08,58883.216281,58883.214143,22355.124126,high


In [110]:
excel_file = "churn_analysis_output.xlsx"  # <-- metti qui il nome del tuo file precedente

with pd.ExcelWriter(excel_file, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    customer_clv.to_excel(writer, sheet_name="customer_clv", index=False)

print(f"Aggiunto/aggiornato lo sheet 'customer_clv' in: {excel_file}")

Aggiunto/aggiornato lo sheet 'customer_clv' in: churn_analysis_output.xlsx


In [111]:
summary_cols = ["clv_bucket"]
if "churn_probability" in clv_df.columns:
    # crea bucket rischio churn in 3 fasce
    clv_df["churn_risk_bucket"] = pd.qcut(clv_df["churn_probability"], q=3, labels=["low", "mid", "high"])
    summary_cols = ["churn_risk_bucket", "clv_bucket"]

churn_clv_matrix = (
    clv_df
    .groupby(summary_cols)
    .agg(
        n_customers=("CustomerID", "count"),
        avg_clv=("clv_12m_adj_clipped", "mean"),
        median_clv=("clv_12m_adj_clipped", "median")
    )
    .reset_index()
    .sort_values(["n_customers"], ascending=False)
)

with pd.ExcelWriter(excel_file, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    churn_clv_matrix.to_excel(writer, sheet_name="churn_x_clv", index=False)

churn_clv_matrix.head(10)

/var/folders/g3/2z65r1r51lx2kpvz456581tc0000gn/T/ipykernel_43478/556134930.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(summary_cols)


,churn_risk_bucket,clv_bucket,n_customers,avg_clv,median_clv
6,high,low,920,119.942617,102.978781
2,low,high,835,5006.184315,3166.806864
4,mid,mid,650,709.988966,662.404781
1,low,mid,279,861.537472,865.007210
5,mid,high,276,2817.343543,2075.849484
3,mid,low,194,202.651275,218.665925
7,high,mid,191,517.849765,456.388105
8,high,high,9,2282.817674,1739.795408
0,low,low,6,206.658334,199.324609


In [132]:
high_high_clients = clv_df[
    (clv_df["churn_risk_bucket"] == "high") &
    (clv_df["clv_bucket"] == "high")
].copy()

high_high_clients.shape

(9, 19)

In [133]:
high_high_clients = high_high_clients[
    [
        "CustomerID",
        "churn_probability",
        "churn_risk_bucket",
        "clv_12m_adj_clipped",
        "clv_bucket"
    ]
].sort_values(
    ["churn_probability", "clv_12m_adj_clipped"],
    ascending=False
)

high_high_clients

,CustomerID,churn_probability,churn_risk_bucket,clv_12m_adj_clipped,clv_bucket
60,12435,0.702753,high,2587.022690,high
267,12770,0.679096,high,1440.149926,high
58,12433,0.660630,high,1719.670573,high
2471,16698,0.656570,high,1638.205465,high
2234,16253,0.648173,high,1409.856580,high
280,12798,0.647300,high,2957.737406,high
2457,16671,0.645997,high,2794.069623,high
711,13572,0.638717,high,1739.795408,high
471,13135,0.638476,high,4258.851401,high


In [134]:
with pd.ExcelWriter(excel_file, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    high_high_clients.to_excel(
        writer,
        sheet_name="priority_high_high",
        index=False
    )

print("Sheet 'priority_high_high' aggiunto correttamente.")

Sheet 'priority_high_high' aggiunto correttamente.


## 6) Segmentazione RFM a quantili

__Step A1 — Crea R, F, M e i punteggi (1–4)__

dividiamo i clienti in quartili (4 gruppi) per ciascuna dimensione.

__Recency__: più basso = meglio → invertiamo lo score

__Frequency__: più alto = meglio

__Monetary__: più alto = meglio

In [136]:
rfm_df = customer_360.copy()

rfm_df["R"] = rfm_df["recency_days"]
rfm_df["F"] = rfm_df["n_invoices"]
rfm_df["M"] = rfm_df["total_revenue"]

In [137]:
rfm_df["R_score"] = pd.qcut(
    rfm_df["R"],
    q=4,
    labels=[4, 3, 2, 1]
).astype(int)

In [138]:
rfm_df["F_score"] = pd.qcut(
    rfm_df["F"].rank(method="first"),
    q=4,
    labels=[1, 2, 3, 4]
).astype(int)

In [139]:
rfm_df["M_score"] = pd.qcut(
    rfm_df["M"].rank(method="first"),
    q=4,
    labels=[1, 2, 3, 4]
).astype(int)

In [140]:
rfm_df["RFM_score"] = (
    rfm_df["R_score"].astype(str) +
    rfm_df["F_score"].astype(str) +
    rfm_df["M_score"].astype(str)
)

In [141]:
def assign_rfm_segment(row):
    r, f, m = row["R_score"], row["F_score"], row["M_score"]

    if r >= 3 and f >= 3 and m >= 3:
        return "Champions"

    if r >= 3 and f >= 2:
        return "Loyal"

    if r <= 2 and f >= 3 and m >= 3:
        return "At Risk High Value"

    if r <= 2 and f <= 2:
        return "Hibernating"

    return "Potential"

rfm_df["rfm_segment"] = rfm_df.apply(assign_rfm_segment, axis=1)

In [142]:
rfm_df["rfm_segment"].value_counts()

rfm_segment
Hibernating           1178
Champions             1030
Loyal                  438
Potential              401
At Risk High Value     313
Name: count, dtype: int64

In [143]:
agg_dict = {
    "CustomerID": "count",
    "R": "mean",
    "F": "mean",
    "M": "mean"
}

if "churn_probability" in rfm_df.columns:
    agg_dict["churn_probability"] = "mean"

if "clv_12m_adj_clipped" in rfm_df.columns:
    agg_dict["clv_12m_adj_clipped"] = "mean"

segment_summary_rfm = (
    rfm_df
    .groupby("rfm_segment")
    .agg(agg_dict)
    .rename(columns={"CustomerID": "n_customers"})
    .sort_values("n_customers", ascending=False)
    .reset_index()
)

segment_summary_rfm

,rfm_segment,n_customers,R,F,M,churn_probability
0,Hibernating,1178,167.956706,1.240238,344.034321,0.705858
1,Champions,1030,24.815534,8.921359,3631.701835,0.216495
2,Loyal,438,35.267123,2.216895,473.968858,0.491506
3,Potential,401,81.000000,1.905237,351.886860,0.587462
4,At Risk High Value,313,114.984026,4.654952,1356.157895,0.418678


In [145]:
customer_360_with_seg = customer_360.merge(
    rfm_df[["CustomerID", "rfm_segment"]],
    on="CustomerID",
    how="left"
)

In [147]:
customer_360_with_seg = customer_360.copy()

In [148]:
clv_df.columns

Index(['CustomerID', 'first_purchase_date', 'last_purchase_date', 'n_invoices',
       'total_revenue', 'avg_revenue_per_line', 'n_products', 'recency_days',
       'tenure_days', 'return_rate', 'churn', 'churn_probability',
       'tenure_months', 'avg_monthly_revenue', 'clv_12m', 'clv_12m_adj',
       'clv_12m_adj_clipped', 'clv_bucket', 'churn_risk_bucket'],
      dtype='object')

In [149]:
customer_master = (
    customer_360
    .merge(
        clv_df[["CustomerID", "clv_12m_adj_clipped"]],
        on="CustomerID",
        how="left"
    )
    .merge(
        rfm_df[["CustomerID", "rfm_segment"]],
        on="CustomerID",
        how="left"
    )
)

In [150]:
customer_master.columns

Index(['CustomerID', 'first_purchase_date', 'last_purchase_date', 'n_invoices',
       'total_revenue', 'avg_revenue_per_line', 'n_products', 'recency_days',
       'tenure_days', 'return_rate', 'churn', 'churn_probability',
       'clv_12m_adj_clipped', 'rfm_segment'],
      dtype='object')

In [151]:
customer_master_view = customer_master[
    [
        "CustomerID",
        "rfm_segment",
        "churn_probability",
        "clv_12m_adj_clipped",
        "recency_days",
        "n_invoices",
        "total_revenue"
    ]
].copy()

In [153]:
excel_file = "churn_analysis_output.xlsx"

with pd.ExcelWriter(excel_file, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    customer_master_view.to_excel(
        writer,
        sheet_name="customer_master",
        index=False
    )

print("Sheet 'customer_master' creato correttamente.")

Sheet 'customer_master' creato correttamente.


In [154]:
def add_rfm_segment(df, rfm_df):
    if "CustomerID" not in df.columns:
        return df

    if "rfm_segment" in df.columns:
        return df

    return df.merge(
        rfm_df[["CustomerID", "rfm_segment"]],
        on="CustomerID",
        how="left"
    )

In [156]:
customer_master_enriched = add_rfm_segment(customer_master, rfm_df)
churn_high_risk_enriched = add_rfm_segment(churn_high_risk, rfm_df)
priority_high_high_enriched = add_rfm_segment(high_high_clients, rfm_df)

In [157]:
excel_file = "churn_analysis_output.xlsx"

with pd.ExcelWriter(excel_file, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    customer_master_enriched.to_excel(
        writer,
        sheet_name="customer_master",
        index=False
    )
    churn_high_risk_enriched.to_excel(
        writer,
        sheet_name="churn_high_risk",
        index=False
    )
    priority_high_high_enriched.to_excel(
        writer,
        sheet_name="priority_high_high",
        index=False
    )

print("rfm_segment aggiunto SOLO ai fogli cliente-level.")

rfm_segment aggiunto SOLO ai fogli cliente-level.
